# Huawei Research France

## Optical network modelling RAMP: cascading regressors

### Predict the output of a 32-channel optical network from its input and side information about the sequence of network components

_Balázs Kégl, Ludovic Dos Santos, Hartmut Hafermann (Huawei Research, Noah's Ark and Optical Laboratories, France)_

## Introduction

Building system models and simulators is a relatively new application area of machine learning. These models can replace expensive experimentation and accelerate design. This year's Huawei France hackathon falls into this category: we will ask participants to learn a model of optical networks from (real) data taken in our Optical Lab. It is a **32-output nonlinear regression problem**, with non-classical challenges:
1. **Transfer learning**: the training and test data will be different: at training we will provide data measured at internal points of the network, whereas we will test the models only for the full cascades, taken on a network with different characteristics from the training network.
2. **Non-standard input**: besides input-output data, we will also provide variable-length side information about the network components which may greatly help the transfer from the training network to the test network.
3. **Scoring metrics** will be a nonstandard measure of the relative prediction error that covers 99% test instances.

### Context

In optical transmission links, information is transmitted through optical fibers. The available transmission bandwidth is equally divided into a fixed number of slots or channels, 32 in this data challenge. Each channel can be used to transmit a beam of laser light at a fixed frequency, modulated by an information carrying signal, such that it occupies its assigned bandwidth. Each of these channels can be used to independently transmit information through the fiber.

Even though modern fibers are highly transparent, the **loss of signal strength** is not negligible and can be as high as a factor of 100 = 20[dB](https://en.wikipedia.org/wiki/Decibel) for 100km of fiber. Fiber transmission links therefore typically consist of a **cascade of alternating optical amplifiers and fibers**. Optical amplifiers compensate for the losses by amplifying the optical signal directly inside the fiber, without the need to convert it into an electrical signal. One of the most widely used type of amplifier is the [Erbium-doped fiber amplifier (EDFA)](https://en.wikipedia.org/wiki/Optical_amplifier#Erbium-doped_optical_fiber_amplifiers). The optical link contain considered here consists of EDFAs, [Single mode fibers (SMFs)](https://en.wikipedia.org/wiki/Single-mode_optical_fiber) and variable optical attenuators (VOAs). The dashed lines in the following figure indicate the positions at which the signal is measured.
<img src="https://raw.githubusercontent.com/ramp-kits/optical_network_modelling/2977eff1ac82349c2a130e89157e2e9db6a79d6a/link.png">

An EDFA implements a **32-input 32-output function**. Computer models of this function are highly useful when telecom providers design  optical networks since these models allow them to optimize the network before building it. In principle, EDFA amplifies its 32 input signals linearly, controlled by two parameters (set by the operator of the network), its nominal gain (the factor by which the signal is amplified on average) and nominal tilt (the factor by which the gain differs for leftmost and rightmost channel). In practice, the **amplification is nonlinear** and it also **depends on which channels are switched on and off**. Generalizing to unseen on/off configurations is important since network operators regularly add or drop channels according to transmission demand. There is no simple physical model of this combinatorial function, thus the aim of our project: explore using machine learning techniques that can be learned on a small subsample of the combinatorial space.

Simlar to an EDFA, a fiber is a nonlinear 32-input 32-output function whose response depends on which channels are switched on and off. The VOAs reduce the power of each channel by the same factor. They simply model the fact that in real optical links losses can occur, for example through poor splices (where two fibers are fused together). Here for simplicty we treat the composition of fiber and VOA as one entity. 

### The data

The data is taken from lab measurements of the channel power in the cascaded link shown in the figure above. For the purpose of this RAMP, we can abstract the transmission cascade as a concatenation of eight modules:
<img src="https://raw.githubusercontent.com/ramp-kits/optical_network_modelling/2977eff1ac82349c2a130e89157e2e9db6a79d6a/cascade.png">
Each module is either an EDFA or an SMF (fiber+VOA), described by a small metadata vector whose format depends on the module type. Each **training instance will be a subcascade** $(i,j)$ with input signal $P_i \in \mathbb{R}^{32}$ and output signal $P_j \in \mathbb{R}^{32}$ and a sequence of module metadata $(m_i, \ldots, m_{j-1})$. Each **test instance will be a full cascade** $(0,8)$ with input signal $P_0 \in \mathbb{R}^{32}$ and output signal $P_8 \in \mathbb{R}^{32}$ and a sequence of module metadata $(m_1, \ldots, m_7)$. 

Each component of the vectors $P_j$ represents the power of the corresponding channel. Since the measured power at the entry and exit of each module differs by large factors, each channel was normalized to its power measured in the configuration where all channels are on. Powers are given on linear scale (not in dB). The values are therefore close to unity.

For each module, the metadata is a vector with two values. For an EDFA, these are the nominal gain and nominal tilt, $m_i=[G_i, T_i]$ for $i$ odd, both measured in dB. The fibers in this dataset are of the same type and length. For the SMF module, the metadata are the factors by which the channels are attenuated in the VOA before or after the fiber, measured in dB: $m_i=[\alpha_i^\text{in}, \alpha_i^\text{out}]$ ($i$ even). The absence of a VOA corresponds to 0 dB. The channel power at the entrance of the fiber can in principle be calculated by deviding the power at the entrance of the module by $10^{\alpha^\text{in}/10}$. The power at the exit of the fiber is correspondingly obtained by multiplying by $10^{\alpha^\text{out}/10}$.

To mimic real-life scenarios, the module metadata of the test cascade will be different from the 
module metadata of the training cascade (although the module types in the sequence will be the same). Each training and test instance will be generated by a random channel on/off configuration, cascaded through the network (so subcascade instances will not be independent). Again, training and test configurations will be different.

### The scoring metrics

We will report a classical RMSE score on the submissions, but the official ranking score will be EM99, which represents an error margin (the uncertainty in the prediction) within which 99% of the test cases fall. Specifically, it is given by the 99th percentile of the relative errors of the test predictions, measured in dB. The metric is only computed over channels which are switched on (non-zero components of the $P_j$).


## Competition rules

The rules may be adjusted before the start of the challenge, November 4 2020.

[//]: # "* Submissions will be trained on a time series of roughly 5000 time steps and tested on a time series of roughly 20000 time steps." 
* The competition will end on November 13, 2020 at 19h UTC (20h in Paris).
* All models will be trained on the same cloud server allowing 4 CPUs (with shared memory of 128GB RAM).
* Participants will be given a total of 20 machine hours. Submissions of a given participant will be ordered by submission timestamp. We will make an attempt to train all submissions, but starting from (and including) the first submission that makes the participant's total training time exceed 20 hours, all submissions will be disqualified from the competition (but can enter into the collaborative phase). Testing time will not count towards the limit. Training time will be displayed on the leaderboard for all submissions, rounded to second. If a submission raises an exception, its training time will not count towards the total.
* There is a timeout of 1 day between submissions.
* Submissions submitted after the end of the competition will not qualify for prizes.
* The public leaderboard will display validation scores running a cross-validation. The official scores will be calculated on the hidden test set and will be published after the closing of the competition. We will rank submissions according to their EM99 score.
* The organizers will do their best so that the provided backend runs flawlessly. We will communicate with participants in case of concerns and will try to resolve all issues, but we reserve the right to make unilateral decisions in specific cases, not covered by this set of minimal rules.
* The organizers reserve the right to disqualify any participant found to violate the fair competitive spirit of the challenge. Possible reasons, without being exhaustive, are multiple accounts, attempts to access the test data, etc.
* The challenge is essentially an individual contest, so there is no way to form official teams. Participants can form teams outside the platform before submitting any model individually, and submit on a single team member's account. However, submitting on one's own and participating in such a team at the same time is against the "no multiple accounts" rule, so, if discovered, may lead to disqualification.
* Participants retain copyright on their submitted code and grant reuse under BSD 3-Clause License.

Participants accept these rules automatically when making a submission at the
Participants accept these rules automatically when making a submission at the RAMP site.